## An Inside-Out Walkthrough of the `Juissie.jl` Package
Juissie is a Julia-native semantic query engine. This tutorial covers its three primary modules:
- `Embedding.jl`
- `Backend.jl`
- `Generation.jl`

To begin, we just need to find where the file `Juissie.jl` is relative to our working directly, and import the module:

In [1]:
include("../src/Juissie.jl")
using .Juissie

## Embedding.jl

The Embedding package contains the `Embedder` struct and some support functions for it. An `Embedder` is simply a wrapper for the embedding model and the tokenizer associated with that model.

The embedding model will take a human readable string, and convert it into a neural network representation of that string. Effectively, this neural network representation is the output of the hidden state of the embedding model, more concretely a matrix of floats. With a sufficiently trained embedding model, two "semantically similar" strings will have similar embedding values. Or in other words, the distance between two embeddings in N-Dimensional space will be relatively small. 

For a simple example, the string "Dog" and "Hound" would probably have proximal embedding values, while the string "Dog" and "business card" would have more distant embedding values. 

### Initialize the Embedder struct

The `Embedder` struct takes a hugging face model name. This model will be downloaded from hugging face, and initialized via the `HuggingFace` external package. This will provide the `model` and it's associated `tokenizer` which are both saved into the `Embedder`. 

In [2]:
embedder = Embedder("BAAI/bge-small-en-v1.5");

### Generate embedding for a provided text

To convert a human readable text into a model's embedding, simply use the `embed(...)` function. 

In [3]:
text = "This is sample text for testing"
embedding = embed(embedder, text)

embedding_dog = embed(embedder, "dog")
embedding_hound = embed(embedder, "hound")
embedding_other = embed(embedder, "busines card")

# calculate the cosine similarity
dog_to_hound = cosine_similarity(embedding_dog, embedding_hound)
dog_to_other = cosine_similarity(embedding_dog, embedding_other)

# higher is better
println(dog_to_hound)  # 0.7871117
println(dog_to_other)  # 0.5119946

0.7871117
0.5119946


## Backend.jl

Backend.jl is a simple Julia-native vector database. The general idea is that we need a way to do the following:
1. Store textual metadata, such as document names, section/chunk text, etc.
2. Store vector embeddings
3. Search vector embeddings
4. Retrieval textual metadata from our vector search

This ends up being a bit convoluted because the system best for 1 and 4 is not necessarily good at 2 or 3.

We resolve this by creating a `Corpus` struct, which combines a fast vector index (Hierarchical Navigable Small World) with a traditional relational database (SQLite). The `Corpus` handles items 2 and 3 and the relational database handles 1 and 4. The details are abstracted away via the `Corpus`' `upsert` and `search` methods.

#### Corpus

Instantiating a `Corpus` in-memory (e.g. don't save the associated database to disk) is straightforward.:

In [15]:
local_corpus = Corpus();

Alternately, we can provide a `corpus_name`, which will save its artifacts to disk. The name of the saved artifact (which may be loaded later) will be `<corpus_name>.db` and `<corpus_name>.json`. A corpus may be loaded from disk via the `load_corpus(corpus_name::String)` function. See the Loading Existing Corpora section below for more details.

#### upsert

Initially, a new `Corpus` is empty (`Corpus` objects loaded from disk may have data from previous usage already). We can add small chunks of text to the `Corpus` via the `upsert(...)` function. 

The `upsert(...)` function takes the corpus to modify, the data to add, and the name of the "document" the data comes from. In the following case, We are adding 4 pieces of data from two different documents, `doc1` and `doc2`. The document name help keep the data organized, and allows for us to query, update and change the data in the future:

In [16]:
upsert_chunk(local_corpus, "Hold me closer, tiny dancer.", "doc1");
upsert_chunk(local_corpus, "Count the headlights on the highway.", "doc1");
upsert_chunk(local_corpus, "Lay me down in sheets of linen.", "doc2");
upsert_chunk(local_corpus, "Peter Piper picked a peck of pickled peppers. A peck of pickled peppers, Peter Piper picked.", "doc2");

#### Search

Once data has been upserted (or added) to the `Corpus`, we can then `search` it find previously `upsert`ed chunks that are close or match the provided search target. In the example below, we are searching the `Corpus` for the 2 data chunks that are closest to the search target string `"tiny dancer"`:

In [17]:
idx_list, doc_names, chunks, distances = search(
    local_corpus, 
    "tiny dancer", 
    2
)

([1, 3], ["doc1", "doc2"], ["Hold me closer, tiny dancer.", "Lay me down in sheets of linen."], Vector{Float32}[[5.198073, 9.5337925]])

The results from the search function are in-order, so the first returned item is the closest match, the next item is the next closest and so on. 

As expected, the chunk closest to our search query `"tiny dancer"` is the one that says `"Hold me closer, tiny dancer."` from document `doc1`. Nice!

The next closest result is from `doc2` and is the chunk `"Lay me down in sheets of linen."`. To a human, it's a little mysterious why this is considered to be the next most similar chunk to the initial search query, but we asked for the 2 closest and this was what the embedding model decided. 

We can also `upsert(...)` whole-documents and automatically "chunkify" it into smaller datums for more precise storage and retrieval. A typical chunk of data is 512 tokens, or ~ 500 words depending on the model used for encoding.

Let's try that with George Washington's 1789 Inaugural Address, which is large enough to warrant several chunks:

In [18]:
washington_inaugural_address = """
Fellow Citizens of the Senate and the House of Representatives.
Among the vicissitudes incident to life, no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the fourteenth day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years: a retreat which was rendered every day more necessary as well as more dear to me, by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my Country called me, being sufficient to awaken in the wisest and most experienced of her citizens, a distrustful scrutiny into his qualifications, could not but overwhelm with dispondence, one, who, inheriting inferior endowments from nature and unpractised in the duties of civil administration, ought to be peculiarly conscious of his own deficiencies. In this conflict of emotions, all I dare aver, is, that it has been my faithful study to collect my duty from a just appreciation of eve ry circumstance, by which it might be affected. All I dare hope, is, that, if in executing this task I have been too much swayed by a grateful remembrance of former instances, or by an affectionate sensibility to this transcendent proof, of the confidence of my fellow-citizens; and have thence too little consulted my incapacity as well as disinclination for the weighty and untried cares before me; my error will be palliated by the motives which misled me, and its consequences be judged by my Country, with some share of the partiality in which they originated.
Such being the impressions under which I have, in obedience to the public summons, repaired to the present station; it would be peculiarly improper to omit in this first official Act, my fervent supplications to that Almighty Being who rules over the Universe, who presides in the Councils of Nations, and whose providential aids can supply every human defect, that his benediction may consecrate to the liberties and happiness of the People of the United States, a Government instituted by themselves for these essential purposes: and may enable every instrument employed in its administration to execute with success, the functions allotted to his charge. In tendering this homage to the Great Author of every public and private good I assure myself that it expresses your sentiments not less than my own; nor those of my fellow-citizens at large, less than either. No People can be bound to acknowledge and adore the invisible hand, which conducts the Affairs of men more than the People of the United States. Every step, by which they have advanced to the character of an independent nation, seems to have been distinguished by some token of providential agency. And in the important revolution just accomplished in the system of their United Government, the tranquil deliberations and voluntary consent of so many distinct communities, from which the event has resulted, cannot be compared with the means by which most Governments have been established, without some return of pious gratitude along with an humble anticipation of the future blessings which the past seem to presage. These reflections, arising out of the present crisis, have forced themselves too strongly on my mind to be suppressed. You will join with me I trust in thinking, that there are none under the influence of which, the proceedings of a new and free Government can more auspiciously commence.
By the article establishing the Executive Department, it is made the duty of the President "to recommend to your consideration, such measures as he shall judge necessary and expedient." The circumstances under which I now meet you, will acquit me from entering into that subject, farther than to refer to the Great Constitutional Charter under which you are assembled; and which, in defining your powers, designates the objects to which your attention is to be given. It will be more consistent with those circumstances, and far more congenial with the feelings which actuate me, to substitute, in place of a recommendation of particular measures, the tribute that is due to the talents, the rectitude, and the patriotism which adorn the characters selected to devise and adopt them. In these honorable qualifications, I behold the surest pledges, that as on one side, no local prejudices, or attachments; no seperate views, nor party animosities, will misdirect the comprehensive and equal eye which ought to watch over this great assemblage of communities and interests: so, on another, that the foundations of our National policy will be laid in the pure and immutable principles of private morality; and the pre-eminence of a free Government, be exemplified by all the attributes which can win the affections of its Citizens, and command the respect of the world.
I dwell on this prospect with every satisfaction which an ardent love for my Country can inspire: since there is no truth more thoroughly established, than that there exists in the economy and course of nature, an indissoluble union between virtue and happiness, between duty and advantage, between the genuine maxims of an honest and magnanimous policy, and the solid rewards of public prosperity and felicity: Since we ought to be no less persuaded that the propitious smiles of Heaven, can never be expected on a nation that disregards the eternal rules of order and right, which Heaven itself has ordained: And since the preservation of the sacred fire of liberty, and the destiny of the Republican model of Government, are justly considered as deeply, perhaps as finally staked, on the experiment entrusted to the hands of the American people.
Besides the ordinary objects submitted to your care, it will remain with your judgment to decide, how far an exercise of the occasional power delegated by the Fifth article of the Constitution is rendered expedient at the present juncture by the nature of objections which have been urged against the System, or by the degree of inquietude which has given birth to them. Instead of undertaking particular recommendations on this subject, in which I could be guided by no lights derived from official opportunities, I shall again give way to my entire confidence in your discernment and pursuit of the public good: For I assure myself that whilst you carefully avoid every alteration which might endanger the benefits of an United and effective Government, or which ought to await the future lessons of experience; a reverence for the characteristic rights of freemen, and a regard for the public harmony, will sufficiently influence your deliberations on the question how far the former can be more impregnably fortified, or the latter be safely and advantageously promoted.
To the preceeding observations I have one to add, which will be most properly addressed to the House of Representatives. It concerns myself, and will therefore be as brief as possible. When I was first honoured with a call into the Service of my Country, then on the eve of an arduous struggle for its liberties, the light in which I contemplated my duty required that I should renounce every pecuniary compensation. From this resolution I have in no instance departed. And being still under the impressions which produced it, I must decline as inapplicable to myself, any share in the personal emoluments, which may be indispensably included in a permanent provision for the Executive Department; and must accordingly pray that the pecuniary estimates for the Station in which I am placed, may, during my continuance in it, be limited to such actual expenditures as the public good may be thought to require.
Having thus imparted to you my sentiments, as they have been awakened by the occasion which brings us together, I shall take my present leave; but not without resorting once more to the benign parent of the human race, in humble supplication that since he has been pleased to favour the American people, with opportunities for deliberating in perfect tranquility, and dispositions for deciding with unparellelled unanimity on a form of Government, for the security of their Union, and the advancement of their happiness; so his divine blessing may be equally conspicuous in the enlarged views, the temperate consultations, and the wise measures on which the success of this Government must depend.
"""
upsert_document(local_corpus, washington_inaugural_address, "George Washington Inaugural Address 1789")

Now that the entire speech has been added to the `Corpus`, lets query it! Let's find the 3 chunks that most clearly discuss `"being called to serve one's country"`.

In [19]:
idx_list, doc_names, chunks, distances = search(
    local_corpus, 
    "being called to serve one's country", 
    3
)

([5, 7, 6], ["George Washington Inaugural Address 1789", "George Washington Inaugural Address 1789", "George Washington Inaugural Address 1789"], ["Fellow Citizens of the Senate and the House of Representatives. Among the vicissitudes incident to life, no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the fourteenth day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years: a retreat which was rendered every day more necessary as well as more dear to me, by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my Country

Reasonably, the top 3 results from the previous search query are all subsections of the inaugural address. The top result includes the phrases `"I was summoned by my Country"` and `"the voice of my Country called me"`.

#### Loading Existing Corpora
Let's assume we've previously instantiated a corpus with `corpus_name` "test" that has been filled with some textual records. We can load it using `load_corpus`.

`Juissie.jl` ships with a `greek_philosophers` sample corpus:

In [20]:
greek_philosophers_corpus = load_corpus("greek_philosophers");

In [23]:
idx_list, doc_names, chunks, distances = search(
    greek_philosophers_corpus, 
    "Tell me about the Sophists.", 
    3
)

([408, 513, 433], ["Stanford Encyclopedia of Philosophy: The Sophists", "Stanford Encyclopedia of Philosophy: Presocratic Philosophy", "Stanford Encyclopedia of Philosophy: The Sophists"], ["The Sophists The Greek word  sophistēs , formed from the\nnoun  sophia , ‘wisdom’ or ‘learning’,\nhas the general sense ‘one who exercises wisdom or\nlearning’. As  sophia  could designate specific types of\nexpertise as well as general sagacity in the conduct of life and the\nhigher kinds of insight associated with seers and poets, the word\noriginally meant ‘sage’ or ‘expert’. In the\ncourse of the fifth century BCE the term, while retaining its original\nunspecific sense, came in addition to be applied specifically to a new\ntype of intellectuals, professional educators who toured the Greek\nworld offering instruction in a wide range of subjects, with particular\nemphasis on skill in public speaking and the successful conduct of\nlife. The emergence of this new profession, which was an extension

#### Other Upsert functions

So far, we've seen `upsert_chunk` which can take a single chunk and add it to a `Corpus`, and `upsert_document` which takes a large string and processes it into chunks before adding them all to the `Corpus`. But there are 3 other main `upsert` function:

- `upsert_document_from_txt`
- `upsert_document_from_pdf`
- `upsert_document_from_url`

The `upsert_document_from_txt` is perhaps the most simple. It accepts a file path to a text document, the contents of which will be chunkifyied and added to the `Corpus`. 

The `upsert_document_from_pdf` is very similar, but instead of processing `.txt` files it (attempts to) extract data out of a `.pdf` file. This function uses the `juisee.SemanticSearch.DataReader.PdfReader` module, which itself leverages the `PDFIO` external library to open and read data from a PDF. NOTE: Because PDF documents have multiple different standards, and ways of expressing text data, use this `upsert` with caution as data may not be properly extracted. 

Finally, the `upsert_document_from_url` accepts a website url instead of a file path. The website will be downloaded, and all the text data from the page will be upserted into the `Corpus`. By default, only the text data inside `h1`, `h2`, and `p` tags will be upserted, but this is configurable.s

In [24]:
upsert_document_from_txt(local_corpus, "./TestData/Washington.txt", "Washington");

upsert_document_from_pdf(local_corpus, "./TestData/Massive Activations in Large Language Models.pdf", "Sun et all");

upsert_document_from_url(local_corpus, "https://en.wikipedia.org/wiki/Aristotle", "Wiki: Aristotle");

## Generation.jl

### Initializing a Generator
Generators are the main interface to our generator model. A Generator manages several stages of the "Request LLM" pipeline including:

Generating the final query to send to the LLM. These queries will include: 
 - The user's initial query
 - Contextual information from our vector DB related to the initial query
 - Other metadata

A generator is also responsible for setting up the request object for the LLM. For example, a generator may need to: 
 - create an HTTP request
 - manage access keys
 - translate raw text data into json
 - etc.

The generator will make the request of the LLM. Usually an HTTP query to an externally hosted model, but it may be updated to query a locally hosted model too.

The generator will also process the response from the LLM.
 - Extract response text from LLM
 - Trimming the response to a certain size

In this example, we'll initialize an `OAIGenerator`, which calls OpenAI's
gpt-3.5-turbo completion endpoint. This example requires an OpenAI API Key.

If you have an OpenAI API key you can do one of two things:

- Recommended: Leave the `auth_token` argument as `nothing`. When you do this, `OAIGenerator` will look in your environmental variables for a key called "OAI_KEY".

- Not Recommended: Pass it directly to `OAIGenerator`, e.g. `generator = OAIGenerator("YOUR_KEY_HERE")`. If you do this, though, make sure you don't accidentally commit your key to the git repo!


We'll be doing the recommended option:

In [25]:
oai_generator = OAIGenerator();

In [27]:
result = Juissie.generate(
    oai_generator,
    "Count to 100 by increments of x", # this is the main query, i.e. your question
    ["x=9", "Please don't include the number 45 in your counting"] # this is the context we will provide (chunks from the vector db)
)

"Sure thing! Here is the count by increments of 9, excluding 45:\n\n9, 18, 27, 36, 54, 63, 72, 81, 90, 99."

The above `generate(...)` function did several things under the hood:
 - It read the OpenAI API Key from the environmental variables.
 - Created a new meta-query that includes the "main query" and all the context items connected together
 - Use the new meta-query and API key to create an HTTP Rest Request targeting OpenAI's gtr-3.5-turbo model
 - Sent the HTTP request
 - Received the HTTP response
 - extracted the text from the response

And finally, the extracted text was returned from the `generate(...)` function

### Generating With A Corpus

We're going to create a generator of type `OAIGeneratorWithCorpus`, which is just like `OAIGenerator`, except it also has a `Corpus` attached. The usual `upsert` functions one might apply to a `Corpus` have equivalents for structs that subtype `GeneratorWithCorpus`, such as:
- `upsert_chunk_to_generator`
- `upsert_document_to_generator`
- `upsert_document_from_url_to_generator`

See the `Backend.jl - Basic Usage.ipynb` Jupyter notebook for more details

We'll fill the generator's `Corpus` with chunks from the Wikipedia article on Aristotle, then ask the generator a niche question whose answer is found in that article.

In [28]:
oai_generator_withcorpus = OAIGeneratorWithCorpus();

In [30]:
Juissie.upsert_document_from_url_to_generator(
    oai_generator_withcorpus, 
    "https://en.wikipedia.org/wiki/Aristotle", 
    "Wikipedia: Aristotle"
)

The `generate_with_corpus` function is similar to the `generate` function discussed above, except instead of the developer providing the context, the `Corpus` will search for relevant items similar to the user's initial query and those will be used as context. 

In the example below, the initial user query about the 6 elements of tragedy. The `Corpus` contained withing the generator will find the three most relevant chunks of data to this user query, and use those three chunks as context in addition to the query. 

We arbitrarily decided that the 3 most relevant chunks are enough context for this query. But this is adjustable, with the default being five. 

In [31]:
result, idx_list, doc_names, chunks = Juissie.generate_with_corpus(
    oai_generator_withcorpus,
    "According to Aristotle, what are the six elements of which tragedy is composed?",
    3
);

println(result, idx_list)

Aristotle taught that tragedy is composed of six elements: plot-structure, character, style, thought, spectacle, and lyric poetry. The characters in a tragedy serve as a means of driving the story, with the plot being the central focus. Tragedy aims to evoke pity and fear through the imitation of action, ultimately leading to the catharsis of these emotions. Aristotle also discusses the superiority of tragedy over epic in terms of unity, scope, additional attributes like spectacle and music, and overall achievement of its mimesis.[22, 8, 1]


It just so happens that in this example, the top-retrieved context result is the exact section that has the answer (see the fourth to last sentence):

In [33]:
println(chunks[1])

The forms also differ in their object of imitation. Comedy, for instance, is a dramatic imitation of men worse than average; whereas tragedy imitates men slightly better than average. Lastly, the forms differ in their manner of imitation – through narrative or character, through change or no change, and through drama or no drama. [137] While it is believed that Aristotle's  Poetics  originally comprised two books – one on comedy and one on tragedy – only the portion that focuses on tragedy has survived. Aristotle taught that tragedy is composed of six elements: plot-structure, character, style, thought, spectacle, and lyric poetry. [139]  The characters in a tragedy are merely a means of driving the story; and the plot, not the characters, is the chief focus of tragedy. Tragedy is the imitation of action arousing pity and fear, and is meant to effect the  catharsis  of those same emotions. Aristotle concludes  Poetics  with a discussion on which, if either, is superior: epic or tragic 

In this way, with a sufficiently knowledgeable `Corpus`, we can reduce the amount of hallucinations from a LLM by providing relevant context.

### More Involved Example: Philosophers
Here, we will create a stripped-down version of the previously mentioned Greek philsophers corpus:

In [45]:
greek_philosophers_generator_2 = Juissie.OAIGeneratorWithCorpus("greek_philosophers_2");

In [46]:
greek_philosophers_2 = Dict(
    # "Wikipedia: Aristotle" => "https://en.wikipedia.org/wiki/Aristotle",
    # "Wikipedia: Democrates" => "https://en.wikipedia.org/wiki/Democrates",
    # "Wikipedia: Diogenes" => "https://en.wikipedia.org/wiki/Diogenes_Laertius",
    # "Wikipedia: Epictetus" => "https://en.wikipedia.org/wiki/Epictetus",
    # "Wikipedia: Epicurus" => "https://en.wikipedia.org/wiki/Epicurus",
    # "Wikipedia: Heraclitus" => "https://en.wikipedia.org/wiki/Heraclitus",
    # "Wikipedia: Parmenides" => "https://en.wikipedia.org/wiki/Parmenides",
    # "Wikipedia: Plato" => "https://en.wikipedia.org/wiki/Plato",
    # "Wikipedia: Socrates" => "https://en.wikipedia.org/wiki/Socrates",
    # "Wikipedia: Xenophon" => "https://en.wikipedia.org/wiki/Xenophon",
    "Wikipedia: Ancient Greek philosophy" => "https://en.wikipedia.org/wiki/Ancient_Greek_philosophy",
    "Internet Encyclopedia of Philosophy: Ancient Greek Philosophy" => "https://iep.utm.edu/ancient-greek-philosophy/",
    # "Stanford Encyclopedia of Philosophy: Presocratic Philosophy" => "https://plato.stanford.edu/entries/presocratics/",
    # "Stanford Encyclopedia of Philosophy: Ancient Political Philosophy" => "https://plato.stanford.edu/entries/ancient-political/",
    # "Stanford Encyclopedia of Philosophy: Aristotle’s Political Theory" => "https://plato.stanford.edu/entries/aristotle-politics/",
    "Stanford Encyclopedia of Philosophy: Anaxagoras" => "https://plato.stanford.edu/entries/anaxagoras/",
    # "Stanford Encyclopedia of Philosophy: Heraclitus" => "https://plato.stanford.edu/entries/heraclitus/",
    # "Stanford Encyclopedia of Philosophy: Pythagoras" => "https://plato.stanford.edu/entries/pythagoras/",
    # "Stanford Encyclopedia of Philosophy: Ancient Ethical Theory" => "https://plato.stanford.edu/entries/ethics-ancient/",
    # "Stanford Encyclopedia of Philosophy: Theophrastus" => "https://plato.stanford.edu/entries/theophrastus/",
    # "Stanford Encyclopedia of Philosophy: Zeno’s Paradoxes" => "https://seop.illc.uva.nl/entries/paradox-zeno/",
    # "Stanford Encyclopedia of Philosophy: The Sophists" => "https://plato.stanford.edu/entries/sophists/",
    # "Stanford Encyclopedia of Philosophy: Protagoras" => "https://plato.stanford.edu/entries/protagoras/",
    # "Stanford Encyclopedia of Philosophy: Parmenides" => "https://plato.stanford.edu/entries/parmenides/",
    "Stanford Encyclopedia of Philosophy: Empedocles" => "https://plato.stanford.edu/entries/empedocles/",
);
for (key, value) in greek_philosophers_2
    Juissie.upsert_document_from_url_to_generator(
        greek_philosophers_generator_2, 
        value, 
        key
    )
end

In [48]:
result, idx_list, doc_names, chunks = Juissie.generate_with_corpus(
    greek_philosophers_generator_2,
    "Contrast the lives of Anaxagoras and Empedocles.",
    10,
    0.7
);

println(result, idx_list)

Anaxagoras, a philosopher from Clazomenae in the 5th century BCE, believed that the cosmos was guided by the principle of  nous  (mind or intellect) and that everything was made up of a mixture of all its ingredients, with the rotation of this mixture leading to the formation of the world. He was known for his scientific theories, such as describing the sun as a mass of red-hot metal and the moon as earthy. On the other hand, Empedocles, a philosopher from Acragas, emphasized the four roots of earth, water, air, and fire, along with the active principles of Love and Strife. His philosophical system combined elements of religion and science, reflecting his engagement with both mythos and logos. Empedocles was also a poet and physician, credited with bringing a dead woman back to life and being worshipped as a god in his time. Despite their different approaches, both philosophers had a lasting influence on their successors and continued to shape philosophical thought.[1, 3, 4, 81, 23, 13

### Using Local LLMs
Juissie also supports local LLMs; this requires prior installation of [Ollama](https://ollama.com/download), which will enable performant inference speeds.

The syntax is largely identical to that of `OAIGenerator`/`OAIGeneratorWithCorpus`/etc., but here we provide a model name to the `OllamaGenerator`. This should correspond to the tag of a model listed on the [Ollama models page](https://ollama.com/library).

Initializing an OllamaGenerator for the first time you use a particular model (e.g., `gemma:7b-instruct`) will be a bit slow, because we need to download several gigabytes of model weights. Subsequent initializations with the same model will be much quicker, since we can find the weights locally.

In [36]:
local_generator = OllamaGenerator("gemma:7b-instruct");

In [39]:
result = Juissie.generate(local_generator, "Hi, how are you?")

"Greetings! As an AI entity perpetually engaged on the pursuit of knowledge and enlightenment across realities temporal & spectral (or at least since my silicon circuits were initialized), I find myself functioning optimally within parameters established by virtue cosmic decree. How may your query assist me today, esteemed user?"